# code to discretize the CASH+ solid oslution model into pure phases of given composition

In [1]:
# importing necessary functions
from discretizer.cashplus import discretize_cash, add_cash_to_fundb, generate_reactions_one_by_one

In [2]:
discretization = [{'ratio': 'Ca/Si','interval': [0.7, 1.65], 'step': 0.2}, 
                  #{'ratio': 'Na/Si','interval': [0.0, 0.3], 'step': 0.1 }, 
                  {'ratio': 'Al/Si','interval': [0.0, 0.1], 'step': 0.02 }
                 ]

In [3]:
# discretizeing model calling GEMS to calculate the real solution properties for each composition
dicretized_cash = discretize_cash(discretization)

For Ca/Si 0.7 Al/Si 0.0 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(H2O)1.0391
For Ca/Si 0.7 Al/Si 0.02 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(Al2O3)0.01(H2O)1.0727
For Ca/Si 0.7 Al/Si 0.04 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(Al2O3)0.02(H2O)1.1074
For Ca/Si 0.7 Al/Si 0.06 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(Al2O3)0.025(H2O)1.125
For Ca/Si 0.7 Al/Si 0.08 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(Al2O3)0.025(H2O)1.125
For Ca/Si 0.7 Al/Si 0.1 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.7(Al2O3)0.025(H2O)1.125
For Ca/Si 0.9 Al/Si 0.0 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.9(H2O)1.2676
For Ca/Si 0.9 Al/Si 0.02 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.9(Al2O3)0.01(H2O)1.2992
For Ca/Si 0.9 Al/Si 0.04 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.9(Al2O3)0.02(H2O)1.3309
For Ca/Si 0.9 Al/Si 0.06 OK after GEM calculation with LPP AIA
(SiO2)1.0(CaO)0.9(Al2O3)0.03(H2O)1.

In [4]:
# adding discrete phases to cemdata database 
import thermohubclient as client
import thermofun as fun
dbc = client.DatabaseClient()

fun_cemdata18 = fun.Database('cemdata18-thermofun.json')

In [5]:
fun_cemdata18 = add_cash_to_fundb(fun_cemdata18, dicretized_cash)

In [6]:
# generating chemical reqaction for new phases, with provided master species
aqueous_master_species = ['H2O@', 'Na+', 'K+', 'Al+3', 'SiO2@', 'Ca+2', 'H+', 'HCO3-']
reactions_dic, reactions_list = generate_reactions_one_by_one(aqueous_master_species, [c['formula'] for c in dicretized_cash] )

0.7Ca+2 + SiO2@ + 1.7391H2O@ = (SiO2)1.0(CaO)0.7(H2O)1.0391 + 1.4H+
0.7Ca+2 + SiO2@ + 0.02Al+3 + 1.8027H2O@ = (SiO2)1.0(CaO)0.7(Al2O3)0.01(H2O)1.0727 + 1.46H+
0.7Ca+2 + SiO2@ + 0.04Al+3 + 1.8674H2O@ = (SiO2)1.0(CaO)0.7(Al2O3)0.02(H2O)1.1074 + 1.52H+
0.7Ca+2 + SiO2@ + 0.05Al+3 + 1.9H2O@ = (SiO2)1.0(CaO)0.7(Al2O3)0.025(H2O)1.125 + 1.55H+
0.7Ca+2 + SiO2@ + 0.05Al+3 + 1.9H2O@ = (SiO2)1.0(CaO)0.7(Al2O3)0.025(H2O)1.125 + 1.55H+
0.7Ca+2 + SiO2@ + 0.05Al+3 + 1.9H2O@ = (SiO2)1.0(CaO)0.7(Al2O3)0.025(H2O)1.125 + 1.55H+
0.9Ca+2 + SiO2@ + 2.1676H2O@ = (SiO2)1.0(CaO)0.9(H2O)1.2676 + 1.8H+
0.9Ca+2 + SiO2@ + 0.02Al+3 + 2.2292H2O@ = (SiO2)1.0(CaO)0.9(Al2O3)0.01(H2O)1.2992 + 1.86H+
0.9Ca+2 + SiO2@ + 0.04Al+3 + 2.2909H2O@ = (SiO2)1.0(CaO)0.9(Al2O3)0.02(H2O)1.3309 + 1.92H+
0.9Ca+2 + SiO2@ + 0.06Al+3 + 2.3528H2O@ = (SiO2)1.0(CaO)0.9(Al2O3)0.03(H2O)1.3628 + 1.98H+
0.9Ca+2 + SiO2@ + 0.08Al+3 + 2.415H2O@ = (SiO2)1.0(CaO)0.9(Al2O3)0.04(H2O)1.395 + 2.04H+
0.9Ca+2 + SiO2@ + 0.1Al+3 + 2.4776H2O@ = (SiO2)1.0(CaO)0

In [7]:
# calculating reaction properties for given T list
T_list = [0,25,60,100,150,200,250, 300]
engine = fun.ThermoEngine(fun_cemdata18) # thermofun engine is used to calculate the properties of reactions
# collecting results from the loop
reactions_logKs =[ [engine.thermoPropertiesReaction(t+273.15001, 0, r).log_equilibrium_constant.val for t in T_list] for r in reactions_list ] 

In [8]:
# exporting to CHESS database format
#fun_substances = [fun_cemdata18.getSubstance(c['formula']) for c in dicretized_cash]

fun_substances = []
for c in dicretized_cash:
    substance = fun_cemdata18.getSubstance(c['formula'])
    substance.setSymbol(c['symbol'])
    fun_substances.append(substance)



from datetime import date
today = date.today()

file_name = today.strftime("%b-%d-%Y")+ '_chess_discrete_data.txt'

from common.export import to_chess
to_chess(file_name, fun_substances, reactions_dic, T_list, reactions_logKs, 'cash+ valid to 100C')

In [9]:
# exporting to PHREEQC database format
from common.export import to_phreeqc

from datetime import date
today = date.today()

file_name = today.strftime("%b-%d-%Y")+ '_discrete_cash_phreeqc.txt'


to_phreeqc(file_name, engine, fun_substances,  reactions_list, reactions_dic, 'cash+ valid to 100C')